In [43]:
from sklearn.model_selection import train_test_split , ShuffleSplit , cross_val_score
from sklearn.linear_model import LinearRegression , Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
import pickle , json
import matplotlib.pyplot as plt

In [2]:
#https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data

df = pd.read_csv("Bengaluru_House_Data.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
df.drop(["area_type", "availability", "society", "balcony"], inplace=True, axis=1)

In [4]:
df.isna().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [5]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df.isna().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [6]:
df.shape

(13246, 5)

## Feature Extraction

In [7]:
df["size"].unique

<bound method Series.unique of 0            2 BHK
1        4 Bedroom
2            3 BHK
3            3 BHK
4            2 BHK
           ...    
13241    5 Bedroom
13242        4 BHK
13243        2 BHK
13244        4 BHK
13245        1 BHK
Name: size, Length: 13246, dtype: object>

In [8]:
def get_bhk_num(x) :
    return int(x.split(' ')[0])

In [9]:
get_bhk_num("2 BHK")

2

In [10]:
df['bhk'] = df['size'].apply(get_bhk_num)
df.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [11]:
df['total_sqft'].describe()

count     13246
unique     2067
top        1200
freq        843
Name: total_sqft, dtype: object

In [12]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [13]:
def get_sqft_num(x):
    try:
        return float(x)
    except:
        n = x.split(" - ")
        if len(n) == 2:
            return ( float(n[0]) + float(n[1]) ) / 2
        else:
            return np.NAN

In [14]:
df['total_sqft'] = df['total_sqft'].apply(get_sqft_num)
df['total_sqft'].describe()

count    13200.000000
mean      1555.302783
std       1237.323445
min          1.000000
25%       1100.000000
50%       1275.000000
75%       1672.000000
max      52272.000000
Name: total_sqft, dtype: float64

In [15]:
df.isna().sum()

location       0
size           0
total_sqft    46
bath           0
price          0
bhk            0
dtype: int64

In [16]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df.isna().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
bhk           0
dtype: int64

In [17]:
df['price_per_sqft'] = df.apply(lambda x: int( x["price"] * 100000 / x["total_sqft"]), axis=1 )
df.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250


In [18]:
df['location'].value_counts(ascending=False)

Whitefield                   532
Sarjapur  Road               392
Electronic City              302
Kanakpura Road               264
Thanisandra                  232
                            ... 
Indiranagar HAL 2nd Stage      1
Maruthi HBCS Layout            1
K R C kothanur                 1
1Channasandra                  1
Abshot Layout                  1
Name: location, Length: 1298, dtype: int64

In [19]:
df.location = df.location.apply(lambda x: x.strip())
location_stats = df['location'].value_counts(ascending=False)
location_stats

Whitefield                   533
Sarjapur  Road               392
Electronic City              304
Kanakpura Road               264
Thanisandra                  235
                            ... 
Rajanna Layout                 1
Subramanyanagar                1
Lakshmipura Vidyaanyapura      1
Malur Hosur Road               1
Abshot Layout                  1
Name: location, Length: 1287, dtype: int64

In [20]:
location_with_10_rows = location_stats[location_stats > 10]

In [21]:
df["location"] = df["location"].apply(lambda x : x if x in location_with_10_rows else 'other')
df['location'].value_counts()

other              2872
Whitefield          533
Sarjapur  Road      392
Electronic City     304
Kanakpura Road      264
                   ... 
Doddaballapur        11
Tindlu               11
Marsur               11
HAL 2nd Stage        11
Kodigehalli          11
Name: location, Length: 241, dtype: int64

## Outliers

In [22]:
df.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250


### Discard rows where total_sqft / bhk is <= 300

In [23]:
df = df[ df['total_sqft'] / df['bhk'] > 300 ]
df.shape

(12274, 7)

### Discard rows where price_per_sqft is outside 1 std

In [24]:
mean = df['total_sqft'].mean()
std = df['total_sqft'].std()

In [25]:
df[ df['total_sqft'] > (mean - std) ]['total_sqft']

0        1056.0
1        2600.0
2        1440.0
3        1521.0
4        1200.0
          ...  
13195    3453.0
13196    3600.0
13197    1141.0
13198    4689.0
13199     550.0
Name: total_sqft, Length: 12274, dtype: float64

In [26]:
df[ df['total_sqft'] < (mean + std) ]['total_sqft']

0        1056.0
1        2600.0
2        1440.0
3        1521.0
4        1200.0
          ...  
13192    1262.0
13193    1345.0
13194    1715.0
13197    1141.0
13199     550.0
Name: total_sqft, Length: 11377, dtype: float64

In [27]:
mean = df['total_sqft'].mean()
std = df['total_sqft'].std()

In [28]:
df['total_sqft'] = df['total_sqft'].apply(lambda x : x if (x > (mean - std)) & (x < (mean + std)) else None)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(11377, 7)

In [29]:
df[ df['total_sqft'] > (mean - std) ]['total_sqft']

0        1056.0
1        2600.0
2        1440.0
3        1521.0
4        1200.0
          ...  
11372    1262.0
11373    1345.0
11374    1715.0
11375    1141.0
11376     550.0
Name: total_sqft, Length: 11377, dtype: float64

In [30]:
df[ df['total_sqft'] < (mean + std) ]['total_sqft']

0        1056.0
1        2600.0
2        1440.0
3        1521.0
4        1200.0
          ...  
11372    1262.0
11373    1345.0
11374    1715.0
11375    1141.0
11376     550.0
Name: total_sqft, Length: 11377, dtype: float64

### Discord rows with bath > bhk + 2

In [31]:
df = df[ df['bath'] < df['bhk'] + 2 ]
df.reset_index(inplace=True, drop=True)
df.shape

(11332, 7)

## Model

In [32]:
df.drop(columns=["size", "price_per_sqft"], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)

In [33]:
df = pd.get_dummies(df, drop_first=True)

In [34]:
X = df.drop("price", axis=1)
y = df['price']

In [35]:
y

0         39.07
1        120.00
2         62.00
3         95.00
4         51.00
          ...  
11327     47.00
11328     57.00
11329    112.00
11330     60.00
11331     17.00
Name: price, Length: 11332, dtype: float64

In [36]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25)

In [37]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [38]:
lr.score(X_test,y_test)

0.5162907372545239

In [39]:
cv = ShuffleSplit(n_splits=5, test_size=0.25, random_state=10)
cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.49707728, 0.52871567, 0.488788  , 0.52131748, 0.53841718])

In [40]:
model_params = {
    'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
}

In [41]:
scores = []

for model_name , model in model_params.items():
    clf = GridSearchCV(model['model'], model['params'], cv=5)
    clf.fit(X,y)
    scores.append([model_name, clf.best_params_, clf.best_score_])
    
scores = pd.DataFrame(scores, columns=["Model", "Parmas", "Score"])
scores

,Model,Parmas,Score
0,linear_regression,{'normalize': True},0.526861
1,lasso,"{'alpha': 1, 'selection': 'random'}",0.445520
2,decision_tree,"{'criterion': 'mse', 'splitter': 'random'}",0.366347


In [42]:
final_model = LinearRegression(normalize=True)
final_model.fit(X_train, y_train)

LinearRegression(normalize=True)

In [54]:
def predict_price(location,sqft,bath,bhk):    
    location = "location_" + location
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return final_model.predict([x])[0]

In [55]:
predict_price( "1st Phase JP Nagar", 2000, 2, 3)

182.19296387918942

In [76]:
predict_price("whitefield", 2000, 2, 3)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [58]:
predict_price("Whitefield", 1800, 3, 4)

120.9919005103316

## Export Model

In [60]:
with open("../model/model.pickle", "wb") as f:
    pickle.dump(final_model,f)

In [61]:
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open('../model/columns.json', 'w') as f:
    f.write(json.dumps(columns))

In [52]:
X

,total_sqft,bath,bhk,location_1st Phase JP Nagar,location_2nd Phase Judicial Layout,location_2nd Stage Nagarbhavi,location_5th Block Hbr Layout,location_5th Phase JP Nagar,location_6th Phase JP Nagar,location_7th Phase JP Nagar,...,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_other
0,1056.0,2.0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600.0,5.0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1440.0,2.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1521.0,3.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1200.0,2.0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11327,1262.0,2.0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11328,1345.0,2.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11329,1715.0,3.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11330,1141.0,2.0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
cols = np.array(columns['data_columns'])

location = ("location_" + "Whitefield").lower()
loc_index = np.where(cols==location)[0]

x = np.zeros(len(X.columns))
x[0] = 2000
x[1] = 2
x[2] = 3
if loc_index >= 0:
    x[loc_index] = 1

final_model.predict([x])[0]

131.48757742542983